In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=4b9aeecb999473651eeffc568a9b79d8eb6a69a2dde001f066a90921ae00577f
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests, json

from pyspark.sql import SparkSession

- We will start by going through the basics of Spark Sessions 

- Create simple Dataframes

- Understand how Spark Metastore works 

- Then bring in the data from the model API end point https://pyrite-ethereal-soccer.glitch.me/docs

- Then learn to do Transformation and loading into files and other locations

In [3]:
spark = SparkSession. \
    builder. \
    appName('SparkOnKaggle'). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 14:22:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [29]:
#Creating spark objects from regular python objects

attendants = [(1, "Smith", "Deo", 1086.0, "Linea"),
             (2, "Kenry", "Jord", 1950.0, "Iran"),
             (3, "Kick", "Boxer", 7550.0, "Kingome"),
             (4, "Gill", "Nome", 1500.0, "New Wold"),
              (5, "Nill", "While", 6500.0, "New Wold"),
              (6, "Will", "Lobe", 18900.0, "Oreleans")
            ]

In [30]:
attendantSDF = spark.createDataFrame(attendants, schema="""attendant_id INT, f_name STRING,
                                     l_name STRING, salary FLOAT, Nationality STRING""")
attendantSDF.show()

+------------+------+------+-------+-----------+
|attendant_id|f_name|l_name| salary|Nationality|
+------------+------+------+-------+-----------+
|           1| Smith|   Deo| 1086.0|      Linea|
|           2| Kenry|  Jord| 1950.0|       Iran|
|           3|  Kick| Boxer| 7550.0|    Kingome|
|           4|  Gill|  Nome| 1500.0|   New Wold|
|           5|  Nill| While| 6500.0|   New Wold|
|           6|  Will|  Lobe|18900.0|   Oreleans|
+------------+------+------+-------+-----------+



In [31]:
attendantSDF.collect()

[Row(attendant_id=1, f_name='Smith', l_name='Deo', salary=1086.0, Nationality='Linea'),
 Row(attendant_id=2, f_name='Kenry', l_name='Jord', salary=1950.0, Nationality='Iran'),
 Row(attendant_id=3, f_name='Kick', l_name='Boxer', salary=7550.0, Nationality='Kingome'),
 Row(attendant_id=4, f_name='Gill', l_name='Nome', salary=1500.0, Nationality='New Wold'),
 Row(attendant_id=5, f_name='Nill', l_name='While', salary=6500.0, Nationality='New Wold'),
 Row(attendant_id=6, f_name='Will', l_name='Lobe', salary=18900.0, Nationality='Oreleans')]

In [7]:
from pyspark.sql.functions import *

In [ ]:
#Three different ways to initiate transformation on the dataframes
#select
attendantSDF.select("f_name","l_name",concat('l_name','f_name').alias('full_name')).show()

In [32]:
#Using withColumn method
attendantSDF.withColumn('new_name',concat('Nationality',lit(' ** '),'f_name')).show()

+------------+------+------+-------+-----------+----------------+
|attendant_id|f_name|l_name| salary|Nationality|        new_name|
+------------+------+------+-------+-----------+----------------+
|           1| Smith|   Deo| 1086.0|      Linea|  Linea ** Smith|
|           2| Kenry|  Jord| 1950.0|       Iran|   Iran ** Kenry|
|           3|  Kick| Boxer| 7550.0|    Kingome| Kingome ** Kick|
|           4|  Gill|  Nome| 1500.0|   New Wold|New Wold ** Gill|
|           5|  Nill| While| 6500.0|   New Wold|New Wold ** Nill|
|           6|  Will|  Lobe|18900.0|   Oreleans|Oreleans ** Will|
+------------+------+------+-------+-----------+----------------+



In [33]:
#Using withColumn method
attendantSDF.selectExpr("salary * 2 AS new_salary", "CONCAT(f_name,l_name) AS new_name").show()

+----------+---------+
|new_salary| new_name|
+----------+---------+
|    2172.0| SmithDeo|
|    3900.0|KenryJord|
|   15100.0|KickBoxer|
|    3000.0| GillNome|
|   13000.0|NillWhile|
|   37800.0| WillLobe|
+----------+---------+



In [34]:
attendantSDF.drop('Nationality').show()

+------------+------+------+-------+
|attendant_id|f_name|l_name| salary|
+------------+------+------+-------+
|           1| Smith|   Deo| 1086.0|
|           2| Kenry|  Jord| 1950.0|
|           3|  Kick| Boxer| 7550.0|
|           4|  Gill|  Nome| 1500.0|
|           5|  Nill| While| 6500.0|
|           6|  Will|  Lobe|18900.0|
+------------+------+------+-------+



In [16]:
type(attendantSDF.drop('Nationality').collect()[0])

pyspark.sql.types.Row

In [35]:
attendantSDF.schema

StructType([StructField('attendant_id', IntegerType(), True), StructField('f_name', StringType(), True), StructField('l_name', StringType(), True), StructField('salary', FloatType(), True), StructField('Nationality', StringType(), True)])

In [36]:
attendantSDF.printSchema()

root
 |-- attendant_id: integer (nullable = true)
 |-- f_name: string (nullable = true)
 |-- l_name: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- Nationality: string (nullable = true)



In [37]:
attendantSDF.write.parquet(path='/kaggle/working/parquet',
                          mode='overwrite',
                          compression='snappy')

In [38]:
new_parquet_read = spark.read.parquet('/kaggle/working/parquet')
new_parquet_read.show()

+------------+------+------+-------+-----------+
|attendant_id|f_name|l_name| salary|Nationality|
+------------+------+------+-------+-----------+
|           4|  Gill|  Nome| 1500.0|   New Wold|
|           5|  Nill| While| 6500.0|   New Wold|
|           6|  Will|  Lobe|18900.0|   Oreleans|
|           2| Kenry|  Jord| 1950.0|       Iran|
|           3|  Kick| Boxer| 7550.0|    Kingome|
|           1| Smith|   Deo| 1086.0|      Linea|
+------------+------+------+-------+-----------+



In [39]:
attendantSDF.coalesce(1).write.mode('append').option("compression","snappy"). \
    option('sep','|').format('csv').save('/kaggle/working/csv/')

In [40]:
attendantSDF.coalesce(1).write.mode('append').option("compression","gzip"). \
    option('sep','|').format('csv').save('/kaggle/working/csv_gzip/')

In [24]:
spark.conf.set("spark.sql.shuffle.partitions",'2')

In [41]:
spark.sql("DROP DATABASE IF EXISTS trial_database")
spark.sql("CREATE DATABASE trial_database")
spark.sql("USE DATABASE trial_database")

DataFrame[]

In [42]:
spark.catalog.currentDatabase()

'trial_database'

In [43]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/kaggle/working/spark-warehouse'),
 Database(name='trial_database', description='', locationUri='file:/kaggle/working/spark-warehouse/trial_database.db')]

In [45]:
attendantSDF.write.saveAsTable("attendants", mode='overwrite',partitionBy='Nationality')

In [46]:
spark.catalog.listTables()

[Table(name='attendants', database='trial_database', description=None, tableType='MANAGED', isTemporary=False)]

In [56]:
spark.sql('DESCRIBE FORMATTED attendants').show(truncate=False)

+----------------------------+-----------------------------------------------------------------+-------+
|col_name                    |data_type                                                        |comment|
+----------------------------+-----------------------------------------------------------------+-------+
|attendant_id                |int                                                              |null   |
|f_name                      |string                                                           |null   |
|l_name                      |string                                                           |null   |
|salary                      |float                                                            |null   |
|Nationality                 |string                                                           |null   |
|# Partition Information     |                                                                 |       |
|# col_name                  |data_type                

In [53]:
spark.catalog.createTable("attendants_new_csv",
                                  path='/kaggle/working/csv/', 
                                 source='csv',
                                 sep='|',
                                 header='true',
                                 inferSchema='true')

DataFrame[1: int, Smith: string, Deo: string, 1086.0: double, Linea: string]

In [54]:
spark.sql('DESCRIBE FORMATTED attendants_new_csv').show(truncate=False)

+----------------------------+--------------------------------------+-------+
|col_name                    |data_type                             |comment|
+----------------------------+--------------------------------------+-------+
|1                           |int                                   |null   |
|Smith                       |string                                |null   |
|Deo                         |string                                |null   |
|1086.0                      |double                                |null   |
|Linea                       |string                                |null   |
|                            |                                      |       |
|# Detailed Table Information|                                      |       |
|Database                    |trial_database                        |       |
|Table                       |attendants_new_csv                    |       |
|Created Time                |Wed Mar 08 15:26:38 UTC 2023      

In [57]:
spark.sql(f"DROP DATABASE IF EXISTS trial_database CASCADE")

DataFrame[]

In [58]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/kaggle/working/spark-warehouse')]

### Starting the ETL process from an API

https://pyrite-ethereal-soccer.glitch.me/docs

https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json

In [59]:
url = 'https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json'
res = requests.request(method='GET',url=url)
data_json = res.json()

In [66]:
main_data = data_json['rows']

In [60]:
spark.stop()

In [61]:
etl_spark = SparkSession. \
    builder. \
    appName('ETL_spark'). \
    getOrCreate()

In [62]:
etl_spark

In [67]:
api_datadf = etl_spark.createDataFrame(data=main_data)

In [68]:
api_datadf.schema

StructType([StructField('download_count', LongType(), True), StructField('project', StringType(), True)])

In [69]:
api_datadf.printSchema()

root
 |-- download_count: long (nullable = true)
 |-- project: string (nullable = true)



In [70]:
# A bit more involved api.. using the pyrite api
pyriteurl = "https://pyrite-ethereal-soccer.glitch.me/biker/v1/deliveryd"
pyriteRes = requests.request(method='GET',url=pyriteurl)
pyriteData = pyriteRes.json()
pyriteData

[{'ID': '0x2318',
  'Delivery_person_ID': 'COIMBRES13DEL01',
  'Type_of_Vehicle': 'electric_scooter',
  'DeliveryPersonAge': 29.5,
  'DeliveryPersonRatings': 4.6,
  'TypeOfVehicle': 2.0,
  'DeliveryPersonID': 2.0},
 {'ID': '0x3474',
  'Delivery_person_ID': 'BANGRES15DEL01',
  'Type_of_Vehicle': 'motorcycle',
  'DeliveryPersonAge': 28.0,
  'DeliveryPersonRatings': 4.6,
  'TypeOfVehicle': 0.0,
  'DeliveryPersonID': 266.0}]

In [72]:
pyriteSDF = etl_spark.createDataFrame(data = pyriteData)
pyriteSDF.show()

+-----------------+----------------+---------------------+------------------+------+-------------+----------------+
|DeliveryPersonAge|DeliveryPersonID|DeliveryPersonRatings|Delivery_person_ID|    ID|TypeOfVehicle| Type_of_Vehicle|
+-----------------+----------------+---------------------+------------------+------+-------------+----------------+
|             29.5|             2.0|                  4.6|   COIMBRES13DEL01|0x2318|          2.0|electric_scooter|
|             28.0|           266.0|                  4.6|    BANGRES15DEL01|0x3474|          0.0|      motorcycle|
+-----------------+----------------+---------------------+------------------+------+-------------+----------------+



In [73]:
pyriteSDF.printSchema()

root
 |-- DeliveryPersonAge: double (nullable = true)
 |-- DeliveryPersonID: double (nullable = true)
 |-- DeliveryPersonRatings: double (nullable = true)
 |-- Delivery_person_ID: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- TypeOfVehicle: double (nullable = true)
 |-- Type_of_Vehicle: string (nullable = true)



In [77]:
#Writing the data to spark catalog
etl_spark.sql("CREATE DATABASE IF NOT EXISTS api_database")
etl_spark.sql("USE DATABASE api_database")
etl_spark.catalog.listDatabases()

[Database(name='api_database', description='', locationUri='file:/kaggle/working/spark-warehouse/api_database.db'),
 Database(name='default', description='default database', locationUri='file:/kaggle/working/spark-warehouse')]

In [78]:
pyriteSDF.write.saveAsTable("pyrite_table")
etl_spark.sql("SELECT * FROM pyrite_table").show(truncate=False)

+-----------------+----------------+---------------------+------------------+------+-------------+----------------+
|DeliveryPersonAge|DeliveryPersonID|DeliveryPersonRatings|Delivery_person_ID|ID    |TypeOfVehicle|Type_of_Vehicle |
+-----------------+----------------+---------------------+------------------+------+-------------+----------------+
|29.5             |2.0             |4.6                  |COIMBRES13DEL01   |0x2318|2.0          |electric_scooter|
|28.0             |266.0           |4.6                  |BANGRES15DEL01    |0x3474|0.0          |motorcycle      |
+-----------------+----------------+---------------------+------------------+------+-------------+----------------+



In [80]:
etl_spark.sql("SHOW TABLES").show()

+------------+------------+-----------+
|   namespace|   tableName|isTemporary|
+------------+------------+-----------+
|api_database|pyrite_table|      false|
+------------+------------+-----------+



In [81]:
etl_spark.sql("SHOW VIEWS").show()

+---------+--------+-----------+
|namespace|viewName|isTemporary|
+---------+--------+-----------+
+---------+--------+-----------+



In [82]:
etl_spark.sql("SHOW DATABASES").show()

+------------+
|   namespace|
+------------+
|api_database|
|     default|
+------------+



In [79]:
etl_spark.catalog.listTables()

[Table(name='pyrite_table', database='api_database', description=None, tableType='MANAGED', isTemporary=False)]

In [83]:
pyriteSDF.coalesce(1).write.csv(path='/kaggle/working/pyrite_csv',
                               sep='|',header=True,
                               mode='overwrite')

In [84]:
!ls /kaggle/working/pyrite_csv/

_SUCCESS  part-00000-fbd2b9b0-c6f1-4d69-80c9-7a0497eab97b-c000.csv


In [85]:
!cat /kaggle/working/pyrite_csv/part-00000-fbd2b9b0-c6f1-4d69-80c9-7a0497eab97b-c000.csv

DeliveryPersonAge|DeliveryPersonID|DeliveryPersonRatings|Delivery_person_ID|ID|TypeOfVehicle|Type_of_Vehicle
29.5|2.0|4.6|COIMBRES13DEL01|0x2318|2.0|electric_scooter
28.0|266.0|4.6|BANGRES15DEL01|0x3474|0.0|motorcycle
